In [1]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, DatePicker, TimePicker, SelectMultiple, Layout, Checkbox
import numpy as np
import pandas as pd
import datetime
from utils.FileBuffer import FileBuffer
import random

%matplotlib inline


In [2]:
out = !ls data/
out = np.array(out)
out

array(['El sistema no puede encontrar la ruta especificada.',
       'n_Fermentación_CL17_-_9-agosto-2023_p4_e8.csv',
       'n_Fermentación_CL18_p6_e20.csv', 'n_Fermentación_DT_p2_e1.csv',
       'n_Fermentación_DT_p2_e4.csv', 'n_Prueba_RIO_p5_e12.csv',
       'n_Prueba_RIO_p5_e13.csv', 'n_Prueba_RIO_p5_e14.csv',
       'n_Prueba_RIO_p5_e15.csv', 'n_Prueba_RIO_p5_e16.csv',
       'n_Prueba_RIO_p5_e17.csv', 'n_Prueba_RIO_p5_e18.csv',
       'n_Prueba_RIO_p5_e19.csv', 'n_Prueba_de_conexión_p3_e2.csv',
       'n_Prueba_de_conexión_p3_e3.csv', 'n_Prueba_de_conexión_p3_e5.csv',
       'n_Prueba_de_conexión_p3_e6.csv', 'n_Prueba_de_conexión_p3_e7.csv',
       'n_Pruebas_3_p7_e21.csv', 'n_Pruebas_3_p8_e22.csv',
       'n_Pruebas_3_p8_e23.csv'], dtype='<U51')

In [3]:
file = out[1]
buffer = FileBuffer(f"data/{file}", batch_size= 10000)
buffer.next()
columns = buffer.get_columns()
strategies = buffer.get_strategies()
filt = buffer.current_data.sort_values("ENT_P103", ascending=False)
filt = filt["ENT_P103"]
ind = filt[filt != float("inf")]

# for x_coord in filt['time']:
#     plt.axvline(x=x_coord, color='red', linestyle='--', label=f'x = {x_coord}')

ind.head()

7547    100.0
7523    100.0
7518    100.0
7519    100.0
7521    100.0
Name: ENT_P103, dtype: float64

In [4]:
def normalize(x,a=0,b=1):
  tmp=x[x != float("inf")]
  denominador = (tmp.max(0) - x.min(0))
  # denominador = denominador if denominador != 0 else 1
  return ( b - a)*(x - x.min(0)) / denominador + a

In [14]:
def plot_data(data, tags, ax, norm):
    x_values = data["time"]
    if norm:
        ax.set_ylim(-0.3, 1.15)
    for i in range(len(tags)):
        y_values = data[tags[i]]
        filt = data[data[tags[i]] == float("inf")]
        for x_coord in filt['time']:
            plt.axvline(x=x_coord, color='red', linestyle='--', label=f'x = {x_coord}')
        if norm:
            y_values = normalize(y_values)
        ax.plot(x_values, y_values, label=tags[i], lw = 2,  marker = "o")
    plt.legend(loc="upper right", bbox_to_anchor=(1.14, 1))

        
def plot_multi_axes(data, tags, ax, norm):
    x_values = data["time"]
    if norm:
        ax.set_ylim(-0.3, 1.15)
    size = len(tags)
    lines = []
    names = []
    i = 0
    eje = ax
    while True:
        if i > 0:
            eje = ax.twinx()
        y_values = data[tags[i]]
        filt = data[data[tags[i]] == float("inf")]
        for x_coord in filt['time']:
            plt.axvline(x=x_coord, color='red', linestyle='--', label=f'x = {x_coord}')
        if norm:
            y_values = normalize(y_values)
        random_color = (random.random()*0.5, random.random(), random.random())
        line, = eje.plot(x_values, y_values, color = random_color, lw = 2, marker = "o")
        eje.set_ylabel(tags[i], color=random_color)
        eje.tick_params(axis='y', labelcolor=random_color)
        lines.append(line)
        names.append(tags[i])
        if i>1:
            eje.spines['right'].set_position(('outward', i*60))
        if i == size-1:
            break
                    
        i+=1
    plt.legend(lines, names, loc='upper left')

    

In [15]:

def generate_plot(start_date, start_hour, end_date, end_hour, tags, strategy, norm, multi):
    start = datetime.datetime.combine(start_date, start_hour)
    end = datetime.datetime.combine(end_date, end_hour)
    tags = np.array(tags).flatten()
    strategy = np.array(strategy).flatten()
    data_filter = buffer.get_by_dates(start, end, strategy[0])
    fig, ax = plt.subplots(1,1,figsize=(20,12))
    if multi:
        plot_multi_axes(data_filter, tags, ax, norm)
    else:
        plot_data(data_filter, tags, ax, norm)
 
        
    
    plt.xticks(rotation=45, fontsize = 15)
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%d-%b\n%H:%M:%S'))
    plt.show()
    print(len(data_filter))

In [16]:


start_date = DatePicker(value=buffer.init_date.date(), description='Inicio: ')
start_hour = TimePicker(value=buffer.init_date.time(), description='Hora:')
end_date = DatePicker(value=buffer.end_date.date(), description='Fin: ')
end_hour = TimePicker(value=buffer.end_date.time(), description='Hora:')
strategies_sele = SelectMultiple( options=strategies, value=[strategies[0]],  description='Strategies:')
tags = SelectMultiple( options=columns, value=[columns[0]],  description='Tags:')
tags.layout = Layout(height = '200px', width = '500px')
strategies_sele.layout = Layout(height = '30px', width = '500px')
checkbox1 = Checkbox(value=False, description="Normalizado")
checkbox2 = Checkbox(value=False, description="MultiEje")

params = {'start_date': start_date, 
          'start_hour': start_hour,
          'end_date': end_date, 
          'end_hour': end_hour, 
          'tags': tags,
          'strategy': strategies_sele,
          'norm':checkbox1,
          'multi':checkbox2}
interactive = widgets.interactive_output(generate_plot, params)
widgets.VBox([
    widgets.HBox([start_date, start_hour, checkbox1, checkbox2]),
    widgets.HBox([end_date, end_hour]),
    widgets.HBox([tags, strategies_sele]),
    interactive
])
